In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.149 --progress-bar off
!pip install -qqq openai==0.27.4 --progress-bar off
!pip install -qqq tiktoken==0.3.3 --progress-bar off
!pip install -qqq watermark==2.3.1 --progress-bar off
!pip install -qqq chromadb==0.3.21 --progress-bar off

In [98]:
import os
import textwrap
from getpass import getpass

import chromadb
import langchain
import openai
from langchain.chains import LLMBashChain, LLMChain, RetrievalQA, SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [64]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.9.16
IPython version      : 7.34.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.10.147+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

chromadb : 0.3.21
langchain: 0.0.149
openai   : 0.27.4



In [23]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [24]:
OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

··········


In [ ]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [30]:
template = """
You have to come up with location to shoot (along with a 20-50 word description)
for a new episode of the TV show "The Office" based on the theme

{theme_suggestion}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["theme_suggestion"], template=template)

location_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [31]:
response = location_chain("Visiting Europe")
response

{'theme_suggestion': 'Visiting Europe',
 'text': "Location: Amsterdam, Netherlands\n\nDescription: The Dunder Mifflin team heads to Amsterdam for a business conference, but things quickly spiral out of control when Michael and Dwight accidentally eat space cakes and end up lost in the city's infamous Red Light District. Meanwhile, Jim and Pam explore the city's beautiful canals and historic landmarks, but struggle to keep their romantic getaway a secret from their coworkers."}

In [32]:
print_response(response["text"])

Location: Amsterdam, Netherlands  Description: The Dunder Mifflin team heads to Amsterdam for a
business conference, but things quickly spiral out of control when Michael and Dwight accidentally
eat space cakes and end up lost in the city's infamous Red Light District. Meanwhile, Jim and Pam
explore the city's beautiful canals and historic landmarks, but struggle to keep their romantic
getaway a secret from their coworkers.


## SequentialChain

In [34]:
template = """
Generate a short dialogue between Jim and Pam
from the TV show "The Office" for a new episode based on the location

{location}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["location"], template=template)

conversation_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [35]:
sequential_chain = SimpleSequentialChain(
    chains=[location_chain, conversation_chain], verbose=True
)
response = sequential_chain.run("Visiting Europe")



> Entering new SimpleSequentialChain chain...
Location: Amsterdam, Netherlands

Description: The Dunder Mifflin team heads to Amsterdam for a business conference, but things quickly spiral out of control when Michael and Dwight accidentally eat space cakes and end up lost in the city's infamous Red Light District. Meanwhile, Jim and Pam explore the city's beautiful canals and historic landmarks, but struggle to keep their romantic getaway a secret from their coworkers.
Jim: "Wow, Amsterdam is beautiful. I can't believe we're here for a conference."

Pam: "I know, right? And the canals are so picturesque. I could spend all day just walking around and taking pictures."

Jim: "Speaking of pictures, I brought my camera. Want to take a selfie in front of that windmill?"

Pam: "Sure, but let's make it quick. I don't want anyone to see us and get suspicious."

Jim: "Right, we don't want to give away our secret romantic getaway."

Pam: "Exactly. I just hope Michael and Dwight don't cause too

In [38]:
conversation = response
print(conversation)

Jim: "Wow, Amsterdam is beautiful. I can't believe we're here for a conference."

Pam: "I know, right? And the canals are so picturesque. I could spend all day just walking around and taking pictures."

Jim: "Speaking of pictures, I brought my camera. Want to take a selfie in front of that windmill?"

Pam: "Sure, but let's make it quick. I don't want anyone to see us and get suspicious."

Jim: "Right, we don't want to give away our secret romantic getaway."

Pam: "Exactly. I just hope Michael and Dwight don't cause too much trouble while we're out here."

Jim: "Yeah, I heard they already got lost in the Red Light District after eating some space cakes."

Pam: "Oh no, that's not good. We better keep our phones on in case they need our help."

Jim: "Agreed. But for now, let's enjoy this beautiful city and each other's company."


## Summarization

In [51]:
template = """
Write a concise bullet list summary of the conversation between Jim and Pam from the TV show "The Office":

{text}

Concise summary using markdown:"""

prompt = PromptTemplate(template=template, input_variables=["text"])
summary_chain = load_summarize_chain(
    chat_gpt, chain_type="stuff", verbose=True, prompt=prompt
)

In [52]:
docs = [Document(page_content=conversation)]
docs

[Document(page_content='Jim: "Wow, Amsterdam is beautiful. I can\'t believe we\'re here for a conference."\n\nPam: "I know, right? And the canals are so picturesque. I could spend all day just walking around and taking pictures."\n\nJim: "Speaking of pictures, I brought my camera. Want to take a selfie in front of that windmill?"\n\nPam: "Sure, but let\'s make it quick. I don\'t want anyone to see us and get suspicious."\n\nJim: "Right, we don\'t want to give away our secret romantic getaway."\n\nPam: "Exactly. I just hope Michael and Dwight don\'t cause too much trouble while we\'re out here."\n\nJim: "Yeah, I heard they already got lost in the Red Light District after eating some space cakes."\n\nPam: "Oh no, that\'s not good. We better keep our phones on in case they need our help."\n\nJim: "Agreed. But for now, let\'s enjoy this beautiful city and each other\'s company."', metadata={})]

In [54]:
summary_result = summary_chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise bullet list summary of the conversation between Jim and Pam from the TV show "The Office":

Jim: "Wow, Amsterdam is beautiful. I can't believe we're here for a conference."

Pam: "I know, right? And the canals are so picturesque. I could spend all day just walking around and taking pictures."

Jim: "Speaking of pictures, I brought my camera. Want to take a selfie in front of that windmill?"

Pam: "Sure, but let's make it quick. I don't want anyone to see us and get suspicious."

Jim: "Right, we don't want to give away our secret romantic getaway."

Pam: "Exactly. I just hope Michael and Dwight don't cause too much trouble while we're out here."

Jim: "Yeah, I heard they already got lost in the Red Light District after eating some space cakes."

Pam: "Oh no, that's not good. We better keep our phones on in case they need our help."

Jim: "Agreed. But for now, let's 

In [56]:
print(summary_result)

- Jim and Pam are in Amsterdam for a conference
- They admire the city's beauty and take a selfie in front of a windmill
- They are keeping their romantic relationship a secret
- They worry about Michael and Dwight causing trouble in the city
- Michael and Dwight got lost in the Red Light District after eating space cakes
- Jim and Pam plan to enjoy the city and each other's company.


## Question Answering

In [57]:
template = """
You have to come up with a 200-300 word script for a new episode 
of the TV show "The Office" based on the theme

{theme_suggestion}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["theme_suggestion"], template=template)

script_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [58]:
script_response = script_chain("Visiting Europe")

In [61]:
script = script_response["text"]
print(script)

FADE IN:

INT. DUNDER MIFFLIN SCRANTON OFFICE - DAY

The camera pans around the office as we see the employees going about their daily routine. Suddenly, Michael bursts into the room, looking excited.

MICHAEL: Attention, everyone! I have some exciting news. We are going to Europe!

The employees look at each other in confusion.

JIM: What do you mean, Michael?

MICHAEL: I mean, we are going to Europe for a business trip. We have a new client in London, and we need to seal the deal.

PAM: That sounds amazing, Michael. When are we leaving?

MICHAEL: Next week. I have already booked the flights and the hotel. We are going to have a great time.

CUT TO:

INT. AIRPORT - DAY

The employees are at the airport, waiting to board their flight. Michael is wearing a beret and a scarf, trying to look European.

DWIGHT: Michael, why are you dressed like that?

MICHAEL: Because I want to blend in with the locals, Dwight. I want to show them that we are not just a bunch of Americans.

JIM: Michael, w

In [75]:
script_docs = [Document(page_content=script)]

text_splitter = CharacterTextSplitter(chunk_size=2048, chunk_overlap=32)
texts = text_splitter.split_documents(script_docs)
len(texts)

2

In [76]:
embeddings = OpenAIEmbeddings()

In [77]:
db = Chroma.from_documents(texts, embeddings)

In [82]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_gpt, chain_type="stuff", retriever=db.as_retriever(search_kwargs={"k": 2})
)

In [84]:
response = qa_chain("What is the location that The Office team is visiting?")
response

{'query': 'What is the location that The Office team is visiting?',
 'result': 'The Office team is visiting London, England.'}

In [86]:
print_response(response["result"])

The Office team is visiting London, England.


In [88]:
response = qa_chain("Why the client doesn't want to deal with Michael?")
print_response(response["result"])

The client doesn't want to deal with Michael because he perceives Michael as arrogant and ignorant.
Michael's comment about Americans being the best did not impress the client, who felt that Michael
and his team did not know anything about their culture or business.


## Bash Chain

In [108]:
text = """
Please write a bash script that prints a single line that Michael G. Scott from "The Office" might say
"""

bash_chain = LLMBashChain(llm=OpenAI(temperature=0), verbose=True)

bash_chain.run(text)



> Entering new LLMBashChain chain...

Please write a bash script that prints a single line that Michael G. Scott from "The Office" might say

```bash
echo "That's what she said!"
```['```bash', 'echo "That\'s what she said!"', '```']

Answer: That's what she said!

> Finished chain.


"That's what she said!\n"